## Imports


In [28]:
import pandas as pd
import joblib
import altair as alt
from gensim.models import Word2Vec
from fydjob.NLPFrame import NLPFrame
import requests

In [2]:
data = joblib.load('/Users/jasminkazi/code/mizzle-toe/find-your-dream-job/fydjob/output/indeed_proc/processed_data.joblib')

In [3]:
data = NLPFrame().df

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jasminkazi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jasminkazi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jasminkazi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Loaded from /Users/jasminkazi/code/mizzle-toe/find-your-dream-job/fydjob/output/nlp_frame.joblib


In [29]:
url = 'http://0.0.0.0:3000'
response = requests.get(url)
response.json()

{'ok': True}

### Exploration


In [4]:
data.shape

(5881, 16)

In [5]:
data.head()

,job_id,job_title,job_text,company,location,job_info,job_link,query_text,source,tag_language,reviews,job_info_tokenized,job_text_tokenized,job_text_tokenized_titlecase,job_title_tokenized,job_text_tokenized_processed
0,1,Data Scientist / Matching Engineer (m/w/d),You are responsible for improvement of Taledo’...,Taledo,Berlin,Data Scientist / Matching Engineer (m/w/d)\nTa...,NULL,data engineer,scrape_json,en,NULL,"[data, scientist, matching, engineer, mwd, tal...","[you, are, responsible, for, improvement, of, ...","[Data, Scientist, Matching, Engineer, mwd, Tal...","[data, scientist, matching, engineer, mwd]","[responsible, improvement, taledos, search, ma..."
1,2,Senior Software Engineer - Data Platform,We are looking for a Senior Software Engineer ...,Zalando SE,Berlin,Senior Software Engineer - Data Platform\nZala...,NULL,data engineer,scrape_json,en,NULL,"[senior, software, engineer, data, platform, z...","[we, are, looking, for, a, senior, software, e...","[Senior, Software, Engineer, Data, Platform, Z...","[senior, software, engineer, data, platform]","[looking, senior, software, engineer, extensiv..."
2,3,Senior Data Engineer (m/w/t),"As a member of the Data Engineering Team, you ...",Quandoo GmbH,Berlin,Senior Data Engineer (m/w/t)\nQuandoo GmbH17 B...,NULL,data engineer,scrape_json,en,NULL,"[senior, data, engineer, mwt, quandoo, gmbh, b...","[as, a, member, of, the, data, engineering, te...","[Senior, Data, Engineer, mwt, Quandoo, GmbH, B...","[senior, data, engineer, mwt]","[member, data, engineering, team, responsible,..."
3,4,Data Engineer (w/m/d),We are digitty.io – an international start-up ...,digitty.io,Berlin,Data Engineer (w/m/d)\ndigitty.io - Berlin,NULL,data engineer,scrape_json,en,NULL,"[data, engineer, wmd, digittyio, berlin]","[we, are, digittyio, an, international, startu...","[Data, Engineer, wmd, digittyio, Berlin]","[data, engineer, wmd]","[digittyio, international, startup, headquarte..."
4,5,Data Engineer,We activate data for our clients by using stat...,Gemma Analytics,Berlin,Data Engineer\nGemma Analytics - Berlin,NULL,data engineer,scrape_json,en,NULL,"[data, engineer, gemma, analytics, berlin]","[we, activate, data, for, our, clients, by, us...","[Data, Engineer, Gemma, Analytics, Berlin]","[data, engineer]","[activate, data, client, using, stateoftheart,..."


In [6]:
data = data.drop(columns=['job_info_tokenized','job_text_tokenized_titlecase', 'job_title_tokenized','job_text_tokenized'])

In [7]:
set(data.columns)

{'company',
 'job_id',
 'job_info',
 'job_link',
 'job_text',
 'job_text_tokenized_processed',
 'job_title',
 'location',
 'query_text',
 'reviews',
 'source',
 'tag_language'}

In [8]:
#set(df.columns)-(set(data.columns))

### No. of...


In [9]:
data['job_title'].count()

5881

In [10]:
data['query_text'].unique()

array(['data engineer', 'business intelligence', 'data scientist',
       'data science', 'NULL'], dtype=object)

In [11]:
print('min: ',data['reviews'].min(), ' max: ', data['reviews'].max())
#data['reviews'].plot()

min:  10.0  max:  NULL


In [12]:
#trying out modifying the df so I don't have a df per chart
source_lang = data['tag_language'].value_counts()
type(source_lang)
source_lang= data.groupby('tag_language')
type(source_lang)

pandas.core.groupby.generic.DataFrameGroupBy

In [13]:
#Lanugauges
data['tag_language'].nunique()
source_lang = pd.DataFrame(data['tag_language'].value_counts()).reset_index()
source_lang = source_lang.rename(columns = {'index':'Posting Language', 'tag_language':'Count'})
type(source_lang)

pandas.core.frame.DataFrame

In [14]:
alt.Chart(source_lang).mark_bar().encode(
    x=alt.X('Count', stack ='normalize'),
    #y='Posting Language',
    color = 'Count'
)


alt.Chart(...)

In [15]:
alt.Chart(source_lang).transform_joinaggregate(
    Total='sum(Count)',
).transform_calculate(
    Percentage="datum.Count / datum.Total"
).mark_bar().encode(
    alt.X('Percentage:Q', axis=alt.Axis(format='.0%')),
    y='Posting Language:N'
)

alt.Chart(...)

In [16]:
#Companies
print('Hiring Companies: ', data['company'].nunique())
source_comp = pd.DataFrame(data['company'].value_counts().reset_index())
source_comp= source_comp.rename(columns = {'index':'Company', 'company':'Count'})
source_comp

Hiring Companies:  2496


,Company,Count
0,Amazon.com,277
1,Microsoft,120
2,Google,57
3,Broad Institute,44
4,McKinsey & Company,40
...,...,...
2491,"CytomX Therapeutics, Inc.",1
2492,ACADIA Pharmaceuticals Inc.,1
2493,Neilson Financial Services,1
2494,Systems Planning and Analysis,1


In [17]:
source_comp = source_comp.nlargest(30, 'Count')

bars = alt.Chart(source_comp).mark_bar().encode(
    x='Count:Q',
    y=alt.Y('Company:N', sort='-x')
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='Count:Q'
)

(bars + text).properties(height=500)

alt.LayerChart(...)

In [18]:
#query_text
print('No of query_text: ',data['query_text'].nunique())
source_job = pd.DataFrame(data['query_text'].value_counts().reset_index())
source_job= source_job.rename(columns = {'index':'Job', 'query_text':'Count'})
source_job

No of query_text:  5


,Job,Count
0,NULL,5262
1,data science,178
2,data engineer,172
3,data scientist,161
4,business intelligence,108


In [19]:
alt.Chart(source_job).mark_bar().encode(
    x='Count',
    y='Job'
)
#rule = alt.Chart(source_job).mark_rule(color='red').encode(
#    x='mean(Count):Q'
#)

alt.Chart(...)

In [21]:
type(data['job_text_tokenized_processed'][0])

list

In [22]:
from fydjob import utils
utils.load_skills()     #loads the skills from JSON file


{'business': ['analyst',
  'report',
  'strategy',
  'consult',
  'distribution',
  'accounting',
  'entrepreneurship',
  'reporting',
  'stakeholder',
  'measurable',
  'analyze',
  'managing',
  'management',
  'administrative',
  'hr',
  'feasibility',
  'solution',
  'analysis',
  'logistic',
  'visualisation',
  'sales',
  'advertising',
  'entrepreneurial',
  'profitable',
  'analyse',
  'efficiency',
  'entrepreneur',
  'strategic',
  'analytical',
  'commercial',
  'olap',
  'consulting',
  'crm',
  'industry',
  'client',
  'kpi',
  'visualization',
  'planning',
  'strategical',
  'analytics',
  'analytic',
  'administration',
  'dashboards',
  'business',
  'marketing',
  'bd',
  'scm'],
 'knowledge': ['statistics',
  'pipeline',
  'etl',
  'reinforcement',
  'geospatial',
  'recurrent',
  'tuning',
  'algorithmical',
  'bayes',
  'methodical',
  'cnn',
  'outlier',
  'signal',
  'pca',
  'databases',
  'graph',
  'knn',
  'predictable',
  'cluster',
  'convolutional',
  'un

In [24]:
# get all the skill sets
categories = utils.load_skills()
categories.keys()
bus_skills = set(utils.load_skills()['business'])
knowledge_skills = set(utils.load_skills()['knowledge'])
code_skills = set(utils.load_skills()['programming'])
soft_skills = set(utils.load_skills()['soft_skills'])


In [26]:
#get skill for the first token
skill_0 = []

for word in data['job_text_tokenized'][0]:
    if word in code_skills:
        skill_0.append(word)
        
skill_0

KeyError: 'job_text_tokenized'

In [ ]:
# count the occurences
from collections import Counter
counter0 = Counter(data['job_text_tokenized'][0])

#occs = {x: counter[x] for x in counter.keys() & set(skill_0)}

bus_occs = {x: counter0[x] for x in counter0.keys() & set(bus_skills)}
code_occs = {x: counter0[x] for x in counter0.keys() & set(code_skills)}
knowledge_occs ={x: counter0[x] for x in counter0.keys() & set(knowledge_skills)}
soft_occs ={x: counter0[x] for x in counter0.keys() & set(soft_skills)}
    
print(bus_occs)
print(code_occs)
print(knowledge_occs)
print(soft_occs)


In [ ]:
jd_0 = set(data['job_text_tokenized'][5])
vacancy = data['job_text_tokenized'][5]
jd_0

In [ ]:
jd_0.intersection(bus_skills)

In [ ]:
jd_0.intersection(code_skills)

In [ ]:
jd_0.intersection(soft_skills)

In [ ]:
jd_0.intersection(knowledge_skills)

In [ ]:
for skill in categories:
    print(skill, jd_0.intersection(categories[skill]))


In [ ]:
intersects_know = jd_0.intersection(knowledge_skills)
intersects_code = jd_0.intersection(code_skills)
intersects_soft = jd_0.intersection(soft_skills)
intersect_bus = jd_0.intersection(bus_skills)

In [ ]:
vacancy.count('xgboost')

In [ ]:
occsurrences_know ={} 
for skill in intersects_know:
    occsurrences_know[skill] = vacancy.count(skill)
occsurrences_know


In [ ]:
occsurrences_code ={} 
for skill in intersects_code:
    occsurrences_code[skill] = vacancy.count(skill)
occsurrences_code

In [ ]:
occsurrences_soft ={} 
for skill in intersects_soft:
    occsurrences_soft[skill] = vacancy.count(skill)
occsurrences_soft

In [ ]:
occsurrences_bus ={} 
for skill in intersects_soft:
    occsurrences_bus[skill] = vacancy.count(skill)
occsurrences_bus


In [ ]:
categories

In [ ]:
for category in categories:
    print(category)

In [ ]:
vacancy = data['job_text_tokenized'][5]
vacancy_set = set(vacancy)
matching_skill_per_category = {}

for category in categories:
    matching_skill_per_category[str(category)]=vacancy_set.intersection(categories[category])
#print(matching_skill_per_category)
    
occ = {}
for category in categories:
    occ[category] = {}
    for s in matching_skill_per_category[str(category)]:
        occ[category][s] =  vacancy.count(s)
print(occ)


In [ ]:
data['job_text_tokenized']

In [ ]:
occ

In [ ]:
for vacancy in data['job_text_tokenized'][:3]:
    set(vacancy)

In [ ]:
type(data['job_text_tokenized'].values)

In [ ]:
categories

In [ ]:
#vacancy = data['job_text_tokenized'][5]


def get_skill_count(vacancy):
    # turn the vacancy into a set to compare the words with defined dictionary
    vacancy_set = set(vacancy)
    # get the matching skills of a job description to the skills defined within different categories of skillsets (tech, bus, softskill...)
    matching_skill_per_category = {}
    for category in categories:
        matching_skill_per_category[str(category)]=vacancy_set.intersection(categories[category])
    
    #count the occurences of every matching skill
    occurrences = {}
    for category in categories:
        occurrences[category] = {}
        for skill in matching_skill_per_category[str(category)]:
            occurrences[category][skill] =  vacancy.count(skill)
    return occurrences

In [ ]:
requirement = {}
for index, vacancy in enumerate(data['job_text_tokenized']):
    #print(vacancy)
    requirement[index]= get_skill_count(vacancy)
    occurence =get_skill_count(vacancy)
    

In [ ]:
requirement

In [ ]:
#things to do for streamlit

# ranking of all skills for streamlit --> agregated view of each categ
# percentage of category per job offer (10% bus, 20% tech...)

In [ ]:
#put all vacancies into one huge list
all_vacancies=[]
for job in data['job_text_tokenized']:
    #print(len(job))
    all_vacancies = all_vacancies + job

In [ ]:
len(all_vacancies)

In [ ]:
vacancy_set = set(all_vacancies)
len(vacancy_set)

In [ ]:

matching_skill_per_category = {}

for category in categories:
    matching_skill_per_category[str(category)]=vacancy_set.intersection(categories[category])
#print(matching_skill_per_category)
occ = {}
for category in categories:
    occ[category] = {}
    for s in matching_skill_per_category[str(category)]:
        occ[category][s] =  all_vacancies.count(s)
print(occ)

In [ ]:
def get_skill_aggr(all_vacancies):
    
    vacancy_set = set(all_vacancies)
    matching_skill_per_category = {}

    for category in categories:
        matching_skill_per_category[str(category)]=vacancy_set.intersection(categories[category])
    
    occ = {}
    for category in categories:
        occ[category] = {}
        for s in matching_skill_per_category[str(category)]:
            occ[category][s] =  all_vacancies.count(s)
    return occ

In [ ]:
all_vacancies=[]
for job in data['job_text_tokenized']:
    #print(len(job))
    all_vacancies = all_vacancies + job
total_occurences = get_skill_aggr(all_vacancies)
total_occurences

In [ ]:
df = pd.DataFrame(total_occurences).reset_index()
df= df.rename(columns = {'index':'skill'})

df

In [ ]:
if df['business'] >0:
    df['category'] = 'business'

In [ ]:
for category in df.columns[1:]:
    #print(df[category])
    if df.loc[df[category] > 0,:]:
              df['category']=category
#df.loc[df['a'] > 10, ['a','c']]

In [ ]:
df_bus = df[['skill','business']].dropna().sort_values('business', ascending = False).rename(columns = {'business':'count'})
df_bus['category']='business'


In [ ]:
df_know = df[['skill','knowledge']].dropna().sort_values('knowledge', ascending = False).rename(columns = {'knowledge':'count'})
df_know['category']='knowledge'


In [ ]:
df_code = df[['skill','programming']].dropna().sort_values('programming', ascending = False).rename(columns = {'programming':'count'})
df_code['category']='programming'


In [ ]:
df_soft = df[['skill','soft_skills']].dropna().sort_values('soft_skills', ascending = False).rename(columns = {'soft_skills':'count'})
df_soft['category']='soft skills'


In [ ]:
df_categ = df_bus.append(df_know).append(df_code).append(df_soft)


In [ ]:
df_categ['category'].unique()

In [ ]:
source = df_categ.nlargest(100, 'count')
categs = np.array(df_categ['category'].unique())
category = np.insert(categs, 0, 'all')


#Dropdownbox
input_dropdown = alt.binding_select(options = category)
selection = alt.selection_single(fields = ['category'], bind = input_dropdown, name= 'Skill ')

alt.Chart(source).mark_bar().encode(
x = ('count:Q'),
y = alt.Y('skill',sort='-x'),
).add_selection(selection).transform_filter(
    selection)


In [ ]:
from vega_datasets import data
movies = alt.UrlData(
    data.movies.url,
    format=alt.DataFormat(parse={"Release_Date":"date"})
)
movies

In [ ]:
movies_df = pd.read_json('https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/movies.json')

In [ ]:
movies_df


In [ ]:
ratings = ['G', 'NC-17', 'PG', 'PG-13', 'R']
genres = ['Action', 'Adventure', 'Black Comedy', 'Comedy',
       'Concert/Performance', 'Documentary', 'Drama', 'Horror', 'Musical',
       'Romantic Comedy', 'Thriller/Suspense', 'Western']

base = alt.Chart(movies, width=200, height=200).mark_point(filled=True).transform_calculate(
    Rounded_IMDB_Rating = "floor(datum.IMDB_Rating)",
    Hundred_Million_Production =  "datum.Production_Budget > 100000000.0 ? 100 : 10",
    Release_Year = "year(datum.Release_Date)"
).transform_filter(
    alt.datum.IMDB_Rating > 0
).transform_filter(
    alt.FieldOneOfPredicate(field='MPAA_Rating', oneOf=ratings)
).encode(
    x=alt.X('Worldwide_Gross:Q', scale=alt.Scale(domain=(100000,10**9), clamp=True)),
    y='IMDB_Rating:Q',
    tooltip="Title:N"
)

genre_dropdown = alt.binding_select(options=genres)
genre_select = alt.selection_single(fields=['Major_Genre'], bind=genre_dropdown, name="Genre")

filter_genres = base.add_selection(
    genre_select
).transform_filter(
    genre_select
).properties(title="Dropdown Filtering")

filter_genres

## Vectorizer


In [ ]:

'''
instanciate a model with the .load method and select the path to the saved .model files
'''

w2v_model = Word2Vec.load("../fydjob/data/models/w2v_model_baseline.model")



In [ ]:
word = ['business', 'dashboard']
df_words= pd.DataFrame(w2v_model.wv.most_similar(word), columns=['Similar word', 'distance']) # works with single words
df_words

In [ ]:
from fydjob.Word2VecPipeline import WordPipeline

model = WordPipeline("../fydjob/data/models/w2v_model_baseline.model")

In [ ]:
word = 'python'
no_rec = 5

model.most_similar_skills(word,n_recommendations= no_rec)